#Predicting sentiment from product reviews

#Fire up GraphLab Create

In [4]:
import graphlab

#Read some product review data

Loading reviews for a set of baby products. 

In [5]:
products = graphlab.SFrame('amazon_baby.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to dsteen01@optonline.net and will expire on October 24, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-4500 - Server binary: C:\Users\david\AppData\Local\Dato\Dato Launcher\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\david\AppData\Local\Temp\graphlab_server_1446999570.log.0
[INFO] GraphLab Server Version: 1.6.1


#Let's explore this data together

Data includes the product name, the review text and the rating of the review. 

In [6]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


#Build the word count vector for each review

In [7]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [8]:
products.head()

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5L, 'stink': 1L,'because': 1L, 'order ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 2L, 'highly': 1L, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1L, 'and':3L, 'love': 2L, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'parents!!':1L, 'all': 2L, 'puppe ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'doll': 1L, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'it': 1L, ' ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'feeding,': 1L, 'and':2L, 'all': 1L, 'right': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'help': 1L,'give': 1L, 'is': 1L, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1L, 'all':1L, 'standarad': 1L, ..."


In [9]:
graphlab.canvas.set_target('ipynb')

In [10]:
products['name'].show()

#Examining the reviews for most-sold product:  'Vulli Sophie the Giraffe Teether'

In [11]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [12]:
len(giraffe_reviews)

785

In [13]:
giraffe_reviews['rating'].show(view='Categorical')

#Build a sentiment classifier

In [14]:
products['rating'].show(view='Categorical')

##Define what's a positive and a negative sentiment

We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment.  Reviews with a rating of 4 or higher will be considered positive, while the ones with rating of 2 or lower will have a negative sentiment.   

In [17]:
#ignore all 3* reviews
products = products[products['rating'] != 3]

In [18]:
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'] >=4

In [19]:
products.head()

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 2L, 'highly': 1L, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1L, 'and':3L, 'love': 2L, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'parents!!':1L, 'all': 2L, 'puppe ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'doll': 1L, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'it': 1L, ' ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'feeding,': 1L, 'and':2L, 'all': 1L, 'right': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'help': 1L,'give': 1L, 'is': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1L, 'all':1L, 'standarad': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1L, 'forget': 1L,'just': 1L, ""daughter ...",1


##Let's train the sentiment classifier

In [20]:
train_data,test_data = products.random_split(.8, seed=0)

In [21]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 219217
PROGRESS: Number of coefficients    : 219218
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 5        | 0.000002  | 2.176176     | 0.841481          | 0.839989            |
PROGRESS: | 2         | 9        | 3.000000  | 3.322503     | 0.947425          | 0.894877            |
PROGRESS: | 3         | 10       | 3.000000  | 3.790819     | 0.92

#Evaluate the sentiment model

In [22]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+----------------+------------------+-------+------+
 |    threshold     |      fpr       |       tpr        |   p   |  n   |
 +------------------+----------------+------------------+-------+------+
 |       0.0        | 0.217432228916 | 0.00603393199404 | 28174 | 5312 |
 | 0.0010000000475  | 0.782567771084 |  0.993966068006  | 28174 | 5312 |
 | 0.00200000009499 | 0.743222891566 |  0.99272378789   | 28174 | 5312 |
 | 0.00300000002608 | 0.720820783133 |  0.992049407255  | 28174 | 5312 |
 | 0.00400000018999 | 0.705384036145 |  0.991552495208  | 28174 | 5312 |
 | 0.00499999988824 | 0.694277108434 |  0.991055583162  | 28174 | 5312 |
 | 0.00600000005215 | 0.684487951807 |  0.990523177398  | 28174 | 5312 |
 | 0.00700000021607 | 0.673945783133 |  0.990132746504  | 28174 | 5312 |
 | 0.00800000037998 | 0.66359186747  |  0.989777809328  | 28174 | 5312 |
 | 0.00899999961257 

In [19]:
sentiment_model.show(view='Evaluation')

#Applying the learned model to understand sentiment for Giraffe

In [23]:
giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews, output_type='probability')

In [21]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'and': 1L, 'all': 1L,'because': 1L, 'it': 1L, ...",0.999513023521
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'and': 1L, 'right': 1L,'help': 1L, 'just': 1L, ...",0.999320678306
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'and': 2L, 'all': 1L,'latex.': 1L, 'being': ...",0.013558811687
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'and': 2L, 'one!': 1L,'all': 1L, 'love': 1L, ...",0.995769474148
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'and': 3L, 'all': 1L,'old': 1L, 'her.': 1L, ...",0.662374415673
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0,"{'and': 6L, 'seven': 1L,'already': 1L, 'love': ...",0.999997148186
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0,"{'and': 4L, 'drooling,':1L, 'love': 1L, 'her.': ...",0.989190989536
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0,"{'and': 3L, 'chew': 1L,""don't"": 1L, 'is': 1L, ...",0.999563518413
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0,"{'chew': 2L, 'because':1L, 'just': 2L, 'what': ...",0.970160542725
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0,"{'and': 2L, 'already':1L, 'in': 1L, 'some': ...",0.195367644588


##Sort the reviews based on the predicted sentiment and explore

In [24]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [25]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5.0,"{'giggles': 1L, 'all':1L, ""violet's"": 2L, ...",1.0
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4.0,"{'peace': 1L, 'month':1L, 'bright': 1L, ...",0.999999999703
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4.0,"{'all': 2L, 'pops': 1L,'existence.': 1L, ...",0.999999999392
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5.0,"{'all': 2L, ""don't"": 1L,'(literally).so': 1L, ...",0.99999999919
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5.0,"{'cute': 1L, 'all': 1L,'reviews.': 2L, 'just': ...",0.999999998657
Vulli Sophie the GiraffeTeether ...,Sophie the Giraffe is theperfect teething toy. ...,5.0,"{'just': 2L, 'both': 1L,'month': 1L, 'ears,': ...",0.999999997108
Vulli Sophie the GiraffeTeether ...,Sophie la giraffe isabsolutely the best toy ...,5.0,"{'and': 5L, 'the': 1L,'all': 1L, 'old': 1L, ...",0.999999995589
Vulli Sophie the GiraffeTeether ...,My 5-mos old son took tothis immediately. The ...,5.0,"{'just': 1L, 'shape': 2L,'mutt': 1L, '""dog': 1L, ...",0.999999995573
Vulli Sophie the GiraffeTeether ...,My nephews and my fourkids all had Sophie in ...,5.0,"{'and': 4L, 'chew': 1L,'all': 1L, 'perfect;': ...",0.999999989527
Vulli Sophie the GiraffeTeether ...,Never thought I'd see myson French kissing a ...,5.0,"{'giggles': 1L, 'all':1L, 'out,': 1L, 'over': ...",0.999999985069


##Most positive reviews for the giraffe

In [24]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [25]:
giraffe_reviews[1]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

##Show most negative reviews for giraffe

In [26]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

In [27]:
giraffe_reviews[-2]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."

In [28]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [27]:
def awesome_count():
    if 'awesome' in dict:
        return 1
    else:
        return 0


In [31]:
products['awesome'] = products['word_count'].apply(awesome_count)

TypeError: awesome_count() takes no arguments (1 given)

In [35]:
temp = products[1]

In [36]:
temp

{'name': 'Annas Dream Full Quilt with 2 Shams',
 'rating': 5.0,
 'review': 'Very soft and comfortable and warmer than it looks...fit the full size bed perfectly...would recommend to anyone looking for this type of quilt',
 'sentiment': 1L,
 'word_count': {'and': 2L,
  'anyone': 1L,
  'bed': 1L,
  'comfortable': 1L,
  'for': 1L,
  'full': 1L,
  'it': 1L,
  'looking': 1L,
  'looks...fit': 1L,
  'of': 1L,
  'perfectly...would': 1L,
  'quilt': 1L,
  'recommend': 1L,
  'size': 1L,
  'soft': 1L,
  'than': 1L,
  'the': 1L,
  'this': 1L,
  'to': 1L,
  'type': 1L,
  'very': 1L,
  'warmer': 1L}}

In [62]:
temp['word_count']

{'and': 2L,
 'anyone': 1L,
 'bed': 1L,
 'comfortable': 1L,
 'for': 1L,
 'full': 1L,
 'it': 1L,
 'looking': 1L,
 'looks...fit': 1L,
 'of': 1L,
 'perfectly...would': 1L,
 'quilt': 1L,
 'recommend': 1L,
 'size': 1L,
 'soft': 1L,
 'than': 1L,
 'the': 1L,
 'this': 1L,
 'to': 1L,
 'type': 1L,
 'very': 1L,
 'warmer': 1L}

In [48]:
x = temp['word_count']

In [49]:
x['quilt']

1L

In [85]:
def awesome_count(y):
    x = y
    if 'awesome' in x:
        return x['awesome']
    else:
        return 0

In [86]:
products['awesome'] = products['word_count'].apply(awesome_count)

In [88]:
products['awesome'].sum()

2002L

In [91]:
def great_count(y):
    x = y
    if 'great' in x:
        return x['great']
    else:
        return 0

In [92]:
products['great'] = products['word_count'].apply(great_count)

In [93]:
products

name,review,rating,word_count,sentiment,awesome
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 2L, 'highly': 1L, ...",1,0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ...",1,0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1L, 'and':3L, 'love': 2L, ...",1,0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'parents!!':1L, 'all': 2L, 'puppe ...",1,0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'doll': 1L, ...",1,0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'it': 1L, ' ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'feeding,': 1L, 'and':2L, 'all': 1L, 'right': ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'help': 1L,'give': 1L, 'is': 1L, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1L, 'all':1L, 'standarad': 1L, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1L, 'forget': 1L,'just': 1L, ""daughter ...",1,0


In [94]:
def fantastic_count(y):
    x = y
    if 'fantastic' in x:
        return x['fantastic']
    else:
        return 0

In [95]:
products['fantastic'] = products['word_count'].apply(fantastic_count)

In [96]:
def amazing_count(y):
    x = y
    if 'amazing' in x:
        return x['amazing']
    else:
        return 0

In [97]:
products['amazing'] = products['word_count'].apply(amazing_count)

In [98]:
def love_count(y):
    x = y
    if 'love' in x:
        return x['love']
    else:
        return 0

In [99]:
products['love'] = products['word_count'].apply(love_count)

In [100]:
products

name,review,rating,word_count,sentiment,awesome
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 2L, 'highly': 1L, ...",1,0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ...",1,0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1L, 'and':3L, 'love': 2L, ...",1,0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'parents!!':1L, 'all': 2L, 'puppe ...",1,0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'doll': 1L, ...",1,0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'it': 1L, ' ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'feeding,': 1L, 'and':2L, 'all': 1L, 'right': ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'help': 1L,'give': 1L, 'is': 1L, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1L, 'all':1L, 'standarad': 1L, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1L, 'forget': 1L,'just': 1L, ""daughter ...",1,0


In [101]:
def horrible_count(y):
    x = y
    if 'horrible' in x:
        return x['horrible']
    else:
        return 0

In [102]:
products['horrible'] = products['word_count'].apply(horrible_count)

In [103]:
def bad_count(y):
    x = y
    if 'bad' in x:
        return x['bad']
    else:
        return 0

In [105]:
products['bad'] = products['word_count'].apply(bad_count)

In [106]:
def terrible_count(y):
    x = y
    if 'terrible' in x:
        return x['terrible']
    else:
        return 0

In [107]:
products['terrible'] = products['word_count'].apply(terrible_count)

In [108]:
def awful_count(y):
    x = y
    if 'awful' in x:
        return x['awful']
    else:
        return 0

In [109]:
products['awful'] = products['word_count'].apply(awful_count)

In [110]:
def wow_count(y):
    x = y
    if 'wow' in x:
        return x['wow']
    else:
        return 0

In [111]:
products['wow'] = products['word_count'].apply(wow_count)

In [112]:
def hate_count(y):
    x = y
    if 'hate' in x:
        return x['hate']
    else:
        return 0

In [130]:
products['hate'] = products['word_count'].apply(hate_count)

In [114]:
products

name,review,rating,word_count,sentiment,awesome
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 2L, 'highly': 1L, ...",1,0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ...",1,0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1L, 'and':3L, 'love': 2L, ...",1,0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'parents!!':1L, 'all': 2L, 'puppe ...",1,0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'doll': 1L, ...",1,0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'it': 1L, ' ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'feeding,': 1L, 'and':2L, 'all': 1L, 'right': ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'help': 1L,'give': 1L, 'is': 1L, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1L, 'all':1L, 'standarad': 1L, ...",1,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1L, 'forget': 1L,'just': 1L, ""daughter ...",1,0


In [117]:
products['awesome'].sum()

2002L

In [118]:
products['great'].sum()

42420.0

In [119]:
products['fantastic'].sum()

873.0

In [121]:
products['amazing'].sum()

1305.0

In [122]:
products['love'].sum()

40277.0

In [123]:
products['horrible'].sum()

659L

In [124]:
products['bad'].sum()

3197L

In [125]:
products['terrible'].sum()

673.0

In [127]:
products['awful'].sum()

345L

In [128]:
products['wow'].sum()

131L

In [131]:
products['hate'].sum()

1057L

In [132]:
train_data,test_data = products.random_split(.8, seed=0)

In [135]:
selected_words

['awesome',
 'great',
 'fantastic',
 'amazing',
 'love',
 'horrible',
 'bad',
 'terrible',
 'awful',
 'wow',
 'hate']

In [134]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [136]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=selected_words,
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 11
PROGRESS: Number of unpacked features : 11
PROGRESS: Number of coefficients    : 12
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 0.080045     | 0.844299          | 0.842842            |
PROGRESS: | 2         | 3        | 0.145088     | 0.844186          | 0.842842            |
PROGRESS: | 3         | 4        | 0.205128     | 0.844276          | 0.843142            |
PROGRESS: | 4         | 5        |

In [137]:
selected_words_model

Class                         : LogisticClassifier

Schema
------
Number of coefficients        : 12
Number of examples            : 133448
Number of classes             : 2
Number of feature columns     : 11
Number of unpacked features   : 11

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : auto
Solver iterations             : 6
Solver status                 : SUCCESS: Optimal solution found.
Training time (sec)           : 0.4053

Settings
--------
Log-likelihood                : 54057.6401

Highest Positive Coefficients
-----------------------------
love                          : 1.3999
(intercept)                   : 1.3673
awesome                       : 1.058
amazing                       : 0.8928
fantastic                     : 0.8913

Lowest Negative Coefficients
----------------------------
terrible                      : -2.0905
horrible                

In [138]:
selected_words_model[‘coefficients’]

SyntaxError: invalid syntax (<ipython-input-138-d8595bb092a0>, line 1)

In [139]:
selected_words_model['coefficients']

name,index,class,value
(intercept),None,1,1.36728315229
awesome,None,1,1.05800888878
great,None,1,0.883937894898
fantastic,None,1,0.891303090304
amazing,None,1,0.892802422508
love,None,1,1.39989834302
horrible,None,1,-1.99651800559
bad,None,1,-0.985827369929
terrible,None,1,-2.09049998487
awful,None,1,-1.76469955631


In [140]:
selected_words_model['coefficients'].sort()

TypeError: sort() takes at least 2 arguments (1 given)

In [141]:
selected_words_model['coefficients'.value()].sort()

AttributeError: 'str' object has no attribute 'value'

In [142]:
x = selected_words_model['coefficients']

In [143]:
x

name,index,class,value
(intercept),None,1,1.36728315229
awesome,None,1,1.05800888878
great,None,1,0.883937894898
fantastic,None,1,0.891303090304
amazing,None,1,0.892802422508
love,None,1,1.39989834302
horrible,None,1,-1.99651800559
bad,None,1,-0.985827369929
terrible,None,1,-2.09049998487
awful,None,1,-1.76469955631


In [144]:
x['value'].sort()

dtype: float
Rows: 12
[-2.090499984872614, -1.9965180055878113, -1.7646995563132248, -1.4091640627563904, -0.9858273699287683, -0.054145012333260044, 0.8839378948979553, 0.8913030903042994, 0.892802422508465, 1.0580088887848045, 1.3672831522930533, 1.3998983430174563]

In [146]:
x.sort('value', ascending = False)

name,index,class,value
love,None,1,1.39989834302
(intercept),None,1,1.36728315229
awesome,None,1,1.05800888878
amazing,None,1,0.892802422508
fantastic,None,1,0.891303090304
great,None,1,0.883937894898
wow,None,1,-0.0541450123333
bad,None,1,-0.985827369929
hate,None,1,-1.40916406276
awful,None,1,-1.76469955631


In [147]:
x.sort('value', ascending = True)

name,index,class,value
terrible,None,1,-2.09049998487
horrible,None,1,-1.99651800559
awful,None,1,-1.76469955631
hate,None,1,-1.40916406276
bad,None,1,-0.985827369929
wow,None,1,-0.0541450123333
great,None,1,0.883937894898
fantastic,None,1,0.891303090304
amazing,None,1,0.892802422508
awesome,None,1,1.05800888878


In [148]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8431419649291376, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  234  |
 |      1       |        0        |  130  |
 |      0       |        1        |  5094 |
 |      1       |        1        | 27846 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

In [149]:
sentiment_model.evaluate(test_data)

{'accuracy': 0.916256305548883, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  1461 |
 |      0       |        1        |  1328 |
 |      0       |        0        |  4000 |
 |      1       |        1        | 26515 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

In [150]:
len(products)

166752

In [153]:
float(products['sentiment'].sum())/float(len(products))

0.8411233448474381

In [160]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [161]:
len(diaper_champ_reviews)

298

In [162]:
diaper_champ_reviews.head()

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,Ok - newsflash. Diapersare just smelly. We've ...,4.0,"{'just': 2L, 'less': 1L,'-': 3L, 'smell- ...",1,0
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1.0,"{'just': 1L, 'less': 1L,'when': 3L, 'over': 1L, ...",0,0
Baby Trend Diaper Champ,Excellent diaper disposalunit. I used it in ...,5.0,"{'control': 1L, 'am': 1L,'it': 1L, 'used': 1L, ...",1,0
Baby Trend Diaper Champ,We love our diaper champ.It is very easy to use ...,5.0,"{'and': 3L, 'over.': 1L,'all': 1L, 'love': 1L, ...",1,0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'just': 1L, 'when': 1L,'both': 1L, 'results': ...",1,0
Baby Trend Diaper Champ,I waited to review thisuntil I saw how it ...,4.0,"{'lysol': 1L, 'all': 1L,'mom.': 1L, 'busy': 1L, ...",1,0
Baby Trend Diaper Champ,I have had a diaper geniefor almost 4 years since ...,1.0,"{'all': 1L, 'bags.': 1L,'just': 1L, ""don't"": 2L, ...",0,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'lysol': 1L, 'all': 2L,'bags.': 1L, 'feedback': ...",1,0
Baby Trend Diaper Champ,I am so glad I got theDiaper Champ instead of ...,5.0,"{'and': 2L, 'all': 1L,'just': 1L, 'is': 2L, ...",1,0
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'hand.': 1L,'(required': 1L, ...",1,0


In [163]:
diaper_champ_reviews['rating'].show(view='Categorical')

In [164]:
sentiment_model.evaluate(diaper_champ_reviews)

{'accuracy': 0.9429530201342282, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |   10  |
 |      0       |        1        |   7   |
 |      0       |        0        |   43  |
 |      1       |        1        |  238  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

In [165]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

In [166]:
diaper_champ_reviews.head()

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,Ok - newsflash. Diapersare just smelly. We've ...,4.0,"{'just': 2L, 'less': 1L,'-': 3L, 'smell- ...",1,0
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1.0,"{'just': 1L, 'less': 1L,'when': 3L, 'over': 1L, ...",0,0
Baby Trend Diaper Champ,Excellent diaper disposalunit. I used it in ...,5.0,"{'control': 1L, 'am': 1L,'it': 1L, 'used': 1L, ...",1,0
Baby Trend Diaper Champ,We love our diaper champ.It is very easy to use ...,5.0,"{'and': 3L, 'over.': 1L,'all': 1L, 'love': 1L, ...",1,0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'just': 1L, 'when': 1L,'both': 1L, 'results': ...",1,0
Baby Trend Diaper Champ,I waited to review thisuntil I saw how it ...,4.0,"{'lysol': 1L, 'all': 1L,'mom.': 1L, 'busy': 1L, ...",1,0
Baby Trend Diaper Champ,I have had a diaper geniefor almost 4 years since ...,1.0,"{'all': 1L, 'bags.': 1L,'just': 1L, ""don't"": 2L, ...",0,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'lysol': 1L, 'all': 2L,'bags.': 1L, 'feedback': ...",1,0
Baby Trend Diaper Champ,I am so glad I got theDiaper Champ instead of ...,5.0,"{'and': 2L, 'all': 1L,'just': 1L, 'is': 2L, ...",1,0
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'hand.': 1L,'(required': 1L, ...",1,0


In [167]:
diaper_champ_reviews.sort('predicted_sentiment', ascending = False)

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'all': 1L, 'less': 1L,""friend's"": 1L, '(whi ...",1,0
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'just': 1L, 'over': 1L,'rweek': 1L, 'sooo': 1L, ...",1,0
Baby Trend Diaper Champ,We researched all of thedifferent types of di ...,4.0,"{'all': 2L, 'just': 4L,""don't"": 2L, 'one,': 1L, ...",1,0
Baby Trend Diaper Champ,My baby is now 8 monthsand the can has been ...,5.0,"{""don't"": 1L, 'when': 1L,'over': 1L, 'soon': 1L, ...",1,0
Baby Trend Diaper Champ,"This is absolutely, byfar, the best diaper ...",5.0,"{'just': 3L, 'money': 1L,'not': 2L, 'mechanism': ...",1,0
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'all': 1L, 'bags.': 1L,'son,': 1L, '(i': 1L, ...",1,0
Baby Trend Diaper Champ,Wow! This is fabulous.It was a toss-up between ...,5.0,"{'and': 4L, '""genie"".':1L, 'since': 1L, ...",1,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'lysol': 1L, 'all': 2L,'bags.': 1L, 'feedback': ...",1,0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'just': 1L, 'when': 1L,'both': 1L, 'results': ...",1,0
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'taller': 1L, 'bags.':1L, 'just': 1L, ""don't"": ...",1,0


In [168]:
selected_words_model.evaluate(diaper_champ_reviews)

{'accuracy': 0.8288590604026845, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |   3   |
 |      0       |        1        |   48  |
 |      1       |        1        |  245  |
 |      0       |        0        |   2   |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

In [169]:
diaper_champ_reviews['selected_words'] = selected_words_model.predict(diaper_champ_reviews, output_type='probability')

In [170]:
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.796940851290673]

In [171]:
diaper_champ_reviews.sort('selected_words', ascending = False)

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,I LOVE LOVE LOVE thisproduct! It is SO much ...,4.0,"{'rating': 1L,'contacted': 1L, 'over': ...",1,0
Baby Trend Diaper Champ,I received my DiaperChamp at my baby shower ...,5.0,"{'bags.': 1L, ""don't"":1L, 'son.': 1L, 'of,': ...",1,0
Baby Trend Diaper Champ,"Love it, love it, loveit! This lives up to ...",5.0,"{'all': 1L, 'already':1L, 'love': 3L, 'have': ...",1,0
Baby Trend Diaper Champ,Works great - no smells.LOVE that it uses reg ...,5.0,"{'and': 2L, 'love': 1L,'garbage': 1L, ...",1,0
Baby Trend Diaper Champ,I love this diaper paleand wouldn't dream of ...,5.0,"{'and': 3L, 'love': 1L,'use.': 1L, 'is': 2L, ...",1,0
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'hand.': 1L,'(required': 1L, ...",1,0
Baby Trend Diaper Champ,I've worked with kidsmore than half my life. ...,5.0,"{'and': 4L, 'genies': 1L,'now': 1L, 'because': ...",1,0
Baby Trend Diaper Champ,"This is absolutely, byfar, the best diaper ...",5.0,"{'just': 3L, 'money': 1L,'not': 2L, 'mechanism': ...",1,0
Baby Trend Diaper Champ,I have a two-year-old sonand I love the Diaper ...,5.0,"{'and': 6L, 'two-year-old': 1L, ""toddler's"": ...",1,0
Baby Trend Diaper Champ,I love this diaper pail!It's so easy to use a ...,5.0,"{'and': 3L, 'this': 1L,'love': 2L, 'being': 1L, ...",1,0


In [172]:
diaper_champ_reviews[0]['review']

"Ok - newsflash.  Diapers are just smelly.  We've had this pail for 2.5 years now.  It was our first and primary one.  There were no major smell problems until after one year, when our son started eating solids.  Also, we change the bag twice weekly as 3 days is about the max for smell-containment.  Around 20-22 months we started shopping for a container that would be less smelly and didn't find one as good.  (We have a cheaper one upstairs which broke immediately and always stunk!)  We finally just put the Diaper Champ in the attic a few months ago and use the cheap one with the flip-up lid - mainly since the cheapo fits inside the cabinet and we didn't notice a big difference in smell-control.  (The most helpful action is to tie the dirty diapers inside a small plastic bag before putting them in the pail.)A couple of our friends have this pail and were pleased until the children started eating solid food and things got stinkier - but that's pretty much the consensus according to many

In [173]:
diaper_champ_reviews[0]['word_count']

{'(the': 1L,
 '(to': 1L,
 '(we': 1L,
 '-': 3L,
 '2.5': 1L,
 '20-22': 1L,
 '3': 1L,
 'a': 6L,
 'about': 1L,
 'according': 1L,
 'action': 1L,
 'after': 1L,
 'ago': 1L,
 'also,': 1L,
 'always': 1L,
 'and': 9L,
 'any': 1L,
 'are': 2L,
 'around': 1L,
 'as': 3L,
 'at': 1L,
 'attic': 1L,
 'babysitters,': 1L,
 'bag': 3L,
 'bags': 1L,
 'be': 1L,
 'before': 1L,
 'big': 1L,
 'broke': 1L,
 'but': 1L,
 'cabinet': 1L,
 'can': 1L,
 'champ': 1L,
 'change': 1L,
 'cheap': 1L,
 'cheaper': 1L,
 'cheapo': 1L,
 'children': 1L,
 'consensus': 1L,
 'container': 1L,
 'convenient!': 1L,
 'couple': 1L,
 'days': 1L,
 'diaper': 1L,
 'diapers': 2L,
 "didn't": 2L,
 'difference': 1L,
 'different': 1L,
 'dirty': 1L,
 'eating': 2L,
 'fact': 1L,
 'few': 1L,
 'finally': 1L,
 'find': 1L,
 'first': 1L,
 'fits': 1L,
 'flip-up': 1L,
 'food': 1L,
 'for': 3L,
 'frequently': 1L,
 'friends': 1L,
 'full.': 1L,
 'gets': 1L,
 'getting': 1L,
 'good.': 1L,
 'got': 1L,
 'grandmothers': 1L,
 'had': 1L,
 'have': 2L,
 'helpful': 1L,
 'how

In [174]:
diaper_champ_reviews[0]

{'amazing': 0.0,
 'awesome': 0L,
 'awful': 0L,
 'bad': 0L,
 'fantastic': 0.0,
 'great': 0.0,
 'hate': 0L,
 'horrible': 0L,
 'love': 0.0,
 'name': 'Baby Trend Diaper Champ',
 'predicted_sentiment': 0.9584435808934214,
 'rating': 4.0,
 'review': "Ok - newsflash.  Diapers are just smelly.  We've had this pail for 2.5 years now.  It was our first and primary one.  There were no major smell problems until after one year, when our son started eating solids.  Also, we change the bag twice weekly as 3 days is about the max for smell-containment.  Around 20-22 months we started shopping for a container that would be less smelly and didn't find one as good.  (We have a cheaper one upstairs which broke immediately and always stunk!)  We finally just put the Diaper Champ in the attic a few months ago and use the cheap one with the flip-up lid - mainly since the cheapo fits inside the cabinet and we didn't notice a big difference in smell-control.  (The most helpful action is to tie the dirty diape